<a href="https://colab.research.google.com/github/dlwogns/ML-DL/blob/main/GoDaddy%20-%20Microbusiness%20Density%20Forecasting/GoDaddy_Dense64.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### train

cflips - 연방정보처리 과정에 따른 각각의 카운티의 고유 넘버. 앞 두자리는 주(state), 뒤 세자리는 카운티. <br>
first day of month - 달의 첫날.<br>
microbusiness density - 각 카운티의 100명당 microbusiness 지수. 이 지표에 사용된 인구수는 2년씩 지연되어있음. (21년 density는 19년의 인구를 사용.) <br>
active - 각 카운티의 microbusiness raw count를 나타냄.<br>

### sample_submission
target -> microbusiness density<br>

### test

row_id, cfips, first day of month 만 제공. -> 시계열

### census starter
관련한 정보들 모음.<br>
이것도 또한 2년의 지연이 있음.<br>
pct_bb -> 카운티 인터넷가입 가정 비율 <br>
pct_college -> 인구수 대비 25세 이상 4년제 대학 학생 비율.<br>

pct foreign born -> 미국 외에서 태어난 사람 비율. 인구대비<br>
pct it workers -> 노동자 중 it산업 종사자 비율<br>
median hh inc -> 카운티 가장 수입 중앙값

# aim to
19년 1월 ~ 22년 10월까지의 정보가 주어지고, 22년 11월 ~ 23년 1월까지 예측

# reference
https://www.kaggle.com/code/kimtaehun/complete-baseline-code-with-various-ml-model<br>
baseline https://www.kaggle.com/code/masahiroichigo/godaddy-microbusiness-density-forecasting-with-tf<br>

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/train.csv")
test = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/test.csv")
census = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/census_starter.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/sample_submission.csv")

In [ ]:
train.nunique()

row_id                   122265
cfips                      3135
county                     1871
state                        51
first_day_of_month           39
microbusiness_density     97122
active                    19193
dtype: int64

In [ ]:
test.nunique()

row_id                25080
cfips                  3135
first_day_of_month        8
dtype: int64

39개의 데이터를 받아서, 8개를 나타내야됨.

census에는 있지만, train에는 없는 것을 제외

In [ ]:
cen = census['cfips'].unique()
tra = train['cfips'].unique()
extra = []
for c in cen:
  if c not in tra:
    extra.append(c)
extra

[2158, 2261, 31117, 46102, 51580, 51640, 51690]

In [ ]:
extra_index = []
for c in extra:
  idx = census[census['cfips'] == c].index[0]
  extra_index.append(idx)
census.iloc[extra_index]

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,pct_foreign_born_2017,pct_foreign_born_2018,pct_foreign_born_2019,pct_foreign_born_2020,pct_foreign_born_2021,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
81,58.4,62.5,65.3,66.1,70.0,2158,3.0,3.3,2.8,2.6,2.3,1.4,1.1,0.5,0.6,0.3,1.7,1.9,1.2,1.1,0.9,36468,35539.0,36754,37358.0,37975.0
92,80.5,79.1,80.4,NaN,NaN,2261,23.1,19.0,16.5,NaN,NaN,4.9,6.3,6.6,NaN,NaN,3.3,3.9,5.3,NaN,NaN,86019,82306.0,79867,NaN,NaN
1712,62.0,59.6,62.8,68.8,78.6,31117,19.6,18.0,17.6,18.5,19.3,0.0,0.0,0.8,0.5,0.4,3.8,0.0,0.0,0.0,0.0,60714,48882.0,48125,51932.0,53750.0
2412,44.8,48.5,45.2,53.0,56.5,46102,9.5,8.7,7.3,6.8,5.9,0.3,0.3,0.3,0.5,0.2,0.5,0.6,0.0,0.0,0.0,27804,30347.0,31997,31423.0,33192.0
2921,59.8,69.0,77.2,80.1,82.9,51580,8.7,10.3,10.4,10.3,8.1,3.4,3.4,3.4,3.0,1.1,2.0,1.9,1.6,0.4,0.4,35546,39432.0,40655,41024.0,41242.0
2928,63.1,66.8,68.5,75.4,81.7,51640,5.7,7.4,8.3,9.7,10.7,9.4,9.9,6.5,7.8,7.9,1.1,0.6,1.5,0.8,0.8,31311,31938.0,33575,35184.0,39808.0
2936,63.4,72.1,73.9,76.6,80.2,51690,13.1,14.1,14.2,14.5,14.9,3.4,2.7,3.8,4.9,5.0,1.4,1.7,1.7,1.6,0.3,33545,33224.0,34371,36166.0,36832.0


In [ ]:
census.drop(extra_index, axis=0, inplace = True)
len(census)

3135

In [ ]:
pd.merge(train, census, how='inner', on='cfips')
# on은 기준열을 잡아주는 것으로, default는 None이고 이 경우에는 index를 기준으로 한다.

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,pct_foreign_born_2017,pct_foreign_born_2018,pct_foreign_born_2019,pct_foreign_born_2020,pct_foreign_born_2021,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,71.1,73.3,76.8,79.7,81.3,14.1,13.5,13.4,12.7,13.9,3.8,4.1,1.7,2.3,1.6,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,71.1,73.3,76.8,79.7,81.3,14.1,13.5,13.4,12.7,13.9,3.8,4.1,1.7,2.3,1.6,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,71.1,73.3,76.8,79.7,81.3,14.1,13.5,13.4,12.7,13.9,3.8,4.1,1.7,2.3,1.6,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,71.1,73.3,76.8,79.7,81.3,14.1,13.5,13.4,12.7,13.9,3.8,4.1,1.7,2.3,1.6,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0


In [ ]:
train["first_day_of_month"] = pd.to_datetime(train["first_day_of_month"])
train = train.sort_values(["cfips", "first_day_of_month"]).reset_index(drop = True)

test["first_day_of_month"] = pd.to_datetime(test["first_day_of_month"])
test = test.sort_values(["cfips", "first_day_of_month"]).reset_index(drop = True)

In [ ]:
import tensorflow as tf

### Time Series Windowing Data
https://albertum.medium.com/preprocessing-time-series-to-windowed-datasets-a464799b1df7

In [ ]:
HORIZON = 8
WINDOW = 8

def get_labelled_window(x, horizon = HORIZON):
  return x[:, :-horizon], x[:, -horizon:]

def make_windows(x, window_size=WINDOW, horizon = HORIZON):
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
  # arange -> 0 ~ argument까지의 일정한 step의 1차원 배열 return.
  # expand_dims -> [1,2], axis=0 -> [[1,2]] [1,2], axis=1 -> [[1],[2]]
  # 차원을 확대하는 것
  # time series를 위한 windowing data를 만들기 위한 
  window_indexes = window_step + np.expand_dims(np.arange(len(x) - (window_size + horizon-1)), axis=0).T
  #print(f'Window indexes: \n{window_indexes, window_indexes.shape}')

  windowed_array = x[window_indexes]

  windows, labels = get_labelled_window(windowed_array, horizon = horizon)

  return windows, labels

In [ ]:
def make_train_test_split(windows, labels, split_size=1):
  split_size = len(windows) - split_size
  train_windows = windows[:split_size]
  train_labels = labels[:split_size]
  test_windows = windows[split_size:]
  test_labels = labels[split_size:]
  return train_windows, test_windows, train_labels, test_labels

In [ ]:
def evaluate_preds(y_true, y_pred):
  y_true = tf.cast(y_true, dtype = tf.float32)
  y_pred = tf.cast(y_pred, dtype = tf.float32)
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)[0]
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)[0]
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)[0]

  return {"mae" : mae.numpy(),
          "mse" : mse.numpy(),
          "rmse" : rmse.numpy(),
          "mape" : mape.numpy()}

In [ ]:
def make_preds(model, input_data):
  forecast = model.predict(input_data)
  return tf.squeeze(forecast)

In [ ]:
def plot_time_series(cfips, timesteps, values, format="-", start=0, end=None, label=None):
    """
    Plots timesteps (a series of points in time) against values (a series of values across timesteps).

    Parameters
    -----------
    timesteps : array of timesteps values
    values : array of values across time
    format : style of plot, default "."
    start : where to start the plot (setting a value will index from start of timesteps & values)
    end : where to end the plot (similar to start but for the end)
    label : label to show on plot about values
    """
    # Plot the series
    plt.plot(timesteps[start:end], values[start:end], format, label=label)
    plt.title(f'Cfips: {cfips}', fontsize=18)
    plt.xlabel("time", fontsize=14)
    plt.ylabel('MicroBusiness Density', fontsize=14)
    if label:
        plt.legend(fontsize=14) # make label bigger
    plt.grid(True)

In [ ]:
IDS = train.cfips.unique()
c = np.random.choice(IDS)
c

5049

In [ ]:
a = train[train['cfips'] == c]
last = a.microbusiness_density.values[-1]
windows, labels = make_windows(a.microbusiness_density.values)
len(windows), len(labels), last

(24, 24, 2.7492819)

In [ ]:
a

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
5187,5049_2019-08-01,5049,Fulton County,Arkansas,2019-08-01,1.670644,161
5188,5049_2019-09-01,5049,Fulton County,Arkansas,2019-09-01,1.805541,174
5189,5049_2019-10-01,5049,Fulton County,Arkansas,2019-10-01,1.753658,169
5190,5049_2019-11-01,5049,Fulton County,Arkansas,2019-11-01,1.753658,169
5191,5049_2019-12-01,5049,Fulton County,Arkansas,2019-12-01,1.743281,168
5192,5049_2020-01-01,5049,Fulton County,Arkansas,2020-01-01,1.730211,167
5193,5049_2020-02-01,5049,Fulton County,Arkansas,2020-02-01,1.771653,171
5194,5049_2020-03-01,5049,Fulton County,Arkansas,2020-03-01,1.719851,166
5195,5049_2020-04-01,5049,Fulton County,Arkansas,2020-04-01,1.813096,175
5196,5049_2020-05-01,5049,Fulton County,Arkansas,2020-05-01,1.833817,177


In [ ]:
train_windows, test_windows, train_labels, test_labels = make_train_test_split(windows, labels)

In [ ]:
train_windows[0]

array([1.6706444, 1.8055412, 1.7536578, 1.7536578, 1.7432811, 1.7302114,
       1.7716535, 1.7198508])

In [ ]:
test_labels

array([[2.4928191, 2.4928191, 2.4620435, 2.5543702, 2.6466968, 2.6774724,
        2.7287648, 2.7492819]])

In [ ]:
train_labels.shape

(23, 8)

In [ ]:
train_windows[0].shape

(8,)

In [ ]:
tf.model = tf.keras.Sequential()

In [ ]:
tf.model.add(tf.keras.layers.Lambda(lambda x : tf.expand_dims(x, axis=0)))
tf.model.add(tf.keras.layers.LSTM(units=64))
tf.model.add(tf.keras.layers.Dense(HORIZON))

In [ ]:
tf.model.build((8,))

ValueError: ignored

In [ ]:
tf.model.summary()

ValueError: ignored

In [ ]:
tf.model.compile(loss='mae', 
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['mae', 'mse'])

In [ ]:
tf.model.fit(x=train_windows, y=train_labels, epochs=100)

Epoch 1/100


ValueError: ignored

random data

In [ ]:
for cfips in IDS[:5]:
  print(cfips)

1001
1003
1005
1007
1009


In [ ]:
np.arange(31, 39).reshape((-1,1))

array([[31],
       [32],
       [33],
       [34],
       [35],
       [36],
       [37],
       [38]])

In [ ]:
x_train = np.arange(39).reshape((-1,1))
x_test = np.arange(31, 39).reshape((-1,1))

In [ ]:
forecasts = []
mape = []
country = []

In [48]:
for cfips in IDS:
  c = cfips
  country.append(c)
  df = train.loc[train.cfips == c]
  last = df.microbusiness_density.values[-1]
  # last?

  windows, labels = make_windows(df.microbusiness_density.values)
  train_windows, test_windows, train_labels, test_labels = make_train_test_split(windows, labels)

  print(train_windows.shape, train_labels.shape)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Lambda(lambda x : tf.expand_dims(x, axis=1)))
  model.add(tf.keras.layers.Dense(128, activation="tanh"))
  model.add(tf.keras.layers.Dense(8))

  model.compile(loss='mae', optimizer = tf.keras.optimizers.Adam(), metrics=['mae', 'mse'])

  model.fit(x=train_windows, y=train_labels, epochs=100)

  preds = make_preds(model, test_windows)

  

  #print(test_labels, preds.type)
  results = evaluate_preds(test_labels, preds)
  m = results['mape']
  mape.append(m)
  #x_test = x_test.reshape(-1)
  print(test_labels, preds)
  m = results['mape']
  mape.append(m)


  plot_time_series(c, x_train, df.microbusiness_density.values, label='train_data')
  plot_time_series(c, x_test, preds, label='forecast')
  plt.show()
  print('##############')

result_df = pd.DataFrame({"Country": country, "tf_dense_MAPE":mape})
  

NameError: ignored

get same value result

In [ ]:
IDS = train.cfips.unique()
x_train = np.arange(31).reshape((-1,1))
x_test = np.arange(31,39).reshape((-1,1))

# Based on the Chris's rule, SMAPE > THRESHOLD -> Use linear regrssion 
mapes = []

for cfips in IDS:
    c = cfips
    
    df = train.loc[train['cfips'] == c]
    last = df['microbusiness_density'].values[-9]


    preds = [last]*8
    test_label = np.expand_dims(np.array(train[train['cfips'] == c]['microbusiness_density'][31:39]), axis=0)
    results = evaluate_preds(test_label, preds)
    m = results['mape']
    forecasts.append(np.array(preds).reshape((-1)))
    mapes.append(m)


result_df['SV_MAPE'] = mapes
result_df['Country'] = country


In [ ]:
result_df.head()

Final

In [ ]:
IDS = train.cfips.unique()
x_train = np.arange(39).reshape((-1,1))
x_test = np.arange(39, 47).reshape((-1,1))

mapes = []
forecasts = []

HORIZON = 8
WINDOW = 8

for cfips in IDS:
  c = cfips
  df = train.loc[train.cfips == c]
  last = df.microbusiness_density.values[-1]

  windows, labels = make_windows(df.microbusiness_density.values)

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Lambda(lambda x : tf.expand_dims(x, axis=1)))
  model.add(tf.keras.layers.Dense(64, activation = "relu"))
  model.add(tf.keras.layers.Dense(8))

  model.compile(loss = 'mae', optimizer = tf.keras.optimizers.Adam(), metrics=['mae', 'mse'])

  model.fit(x=windows, y = labels, epochs = 100, batch_size = 256, verbose = 0)

  forecast = model.predict(tf.expand_dims(labels[-1], axis=0))
  shift = last - np.squeeze(forecast)[0]
  preds = np.squeeze(forecast) + shift
  forecasts.append(preds)

  plot_time_series(c, x_train, df.microbusiness_density.values, label='raw')
  plot_time_series(c, x_test, preds, label='forecast')
  plt.show()

  print(forecasts[-1])


In [ ]:
forecasts[:5]

[array([3.4638555, 3.5554578, 3.492649 , 3.5339296, 3.5340295, 3.534905 ,
        3.520107 , 3.5961661], dtype=float32),
 array([8.359798 , 8.286399 , 8.246088 , 8.4585495, 8.316689 , 8.606373 ,
        8.486988 , 8.526772 ], dtype=float32),
 array([1.2320744, 1.2547454, 1.2448016, 1.2714268, 1.2762471, 1.2996291,
        1.2881149, 1.2978786], dtype=float32),
 array([1.28724  , 1.260013 , 1.2663271, 1.2747561, 1.2617592, 1.261281 ,
        1.2622482, 1.2837539], dtype=float32),
 array([1.8317825, 1.8569071, 1.8654699, 1.8674748, 1.8789757, 1.9105934,
        1.89745  , 1.929141 ], dtype=float32)]

In [ ]:
t = forecasts

In [ ]:
t = np.array(t).reshape((-1))

In [ ]:
test['microbusiness_density'] = np.array(forecasts).reshape((-1))

sub = test[['row_id', 'microbusiness_density']]
sub.to_csv('submission.csv', index=False)
print(sub.shape)
sub.head()

(25080, 2)


,row_id,microbusiness_density
0,1001_2022-11-01,3.463856
1,1001_2022-12-01,3.555458
2,1001_2023-01-01,3.492649
3,1001_2023-02-01,3.533930
4,1001_2023-03-01,3.534029
